## Bonds and Interest rates
Ismael Oulkhir


Mail: oulkhir.ismael@gmail.com

LinkedIn: https://www.linkedin.com/in/ismail-oulkhir/


We assume that, for every fixed T > 0, the forward rate $f (·,T)$ has a stochastic differential which under the objective measure P is given by : 

$df(t,T) = α(t,T)dt + σ(t,T)dW_{t}^{P} $ <br>
$ f(0, T ) = f^*(0, T ),$

Having the initial condition as the observed bond prices at t=0 allows to directly have a perfect fit between theoretical model and observed forward curve.

In [1]:
import numpy as np
import math as sqrt
import random as std


## Libor Market Model

The LIBOR Market Model describes the evolution of forward rates under the risk-neutral measure. The dynamics of the forward rate Fk(t)Fk​(t) (for the period $[Tk,Tk+1]$ are given by :

$dF_k(t)=μ_k​(t)F_k(t)dt+σ_k​(t)F_k​(t)dW_t​$

- $μ_k​(t)$ : Drift term (ensures no-arbitrage).
- $σ_k(t)$ : Volatility of the forward rate.
- $dWt$ : Wiener process (randomness).



In [10]:
import numpy as np

def one_factor_LIBOR_Market_Model(time_step, maturity, zero_curve, forward_rate_volatilities, N):
    # Calculate the number of steps
    steps = int(maturity / time_step) + 1

    # Ensure zero_curve and forward_rate_volatilities have the correct length
    if len(zero_curve) < steps:
        raise ValueError("zero_curve must have at least `steps` elements.")
    if len(forward_rate_volatilities) < steps - 1:
        raise ValueError("forward_rate_volatilities must have at least `steps - 1` elements.")

    # Time grid
    t = np.zeros(steps)
    time = 0
    for i in range(steps):
        t[i] = time
        time += time_step

    # Time step array
    Delta = np.full((steps - 1), time_step)

    # Zero-coupon bond prices
    B_0 = np.zeros(steps)
    for i in range(steps):
        B_0[i] = 1 / (1 + zero_curve[i]) ** (i + 1)

    # Initial forward rates from zero-coupon bond prices
    forward_rate_from_zero = np.zeros((steps - 1, steps - 1))
    for i in range(steps - 1):
        forward_rate_from_zero[i][0] = (1 / Delta[i]) * (B_0[i] / B_0[i + 1] - 1)

    # Monte Carlo simulation
    forward_rate_mc = np.zeros((steps - 1, steps - 1))
    for n in range(N):
        forward_rate = np.zeros((steps - 1, steps - 1))
        for i in range(steps - 1):
            forward_rate[i][0] = forward_rate_from_zero[i][0]
        for k in range(1, steps - 1):
            for j in range(k):
                sum1 = 0
                for i in range(j + 1, k + 1):
                    sum1 += (
                        Delta[i] * forward_rate[i][j] * forward_rate_volatilities[i - j - 1] * forward_rate_volatilities[k - j - 1]
                    ) / (1 + Delta[i] * forward_rate[i][j])
                e = np.random.standard_normal()
                forward_rate[k][j + 1] = forward_rate[k][j] * np.exp(
                    (sum1 - forward_rate_volatilities[k - j - 1] ** 2 / 2) * Delta[j] + forward_rate_volatilities[k - j - 1] * e * np.sqrt(Delta[j])
                )
        forward_rate_mc += forward_rate

    # Average across all paths
    forward_rate_mc = forward_rate_mc / N
    return forward_rate_mc

In [ ]:
# Input data
zero_curve = np.array([0.0074, 0.0074, 0.0077, 0.0082, 0.0088, 0.0094, 0.0101, 0.0108, 0.0116, 0.0123, 0.0131])
forward_rate_volatilities = np.array([0.155, 0.20636739, 0.17209861, 0.17219933, 0.1524579, 0.14147795, 0.12977111, 0.13810532, 0.13595499, 0.13398418])

# Parameters
time_step = 0.25  # Quarterly time step
maturity = len(zero_curve) * time_step  # Total maturity
N = 1000  # Number of Monte Carlo paths

# Run the simulation
simulated_forward_rates = one_factor_LIBOR_Market_Model(time_step, maturity, zero_curve, forward_rate_volatilities, N)

# Print the results
print("Simulated Forward Rates:")
print(simulated_forward_rates)